In [1]:
# Для мониторинга выполнения циклов
from tqdm import tqdm_notebook, tqdm

# Обработка HTML 
from bs4 import BeautifulSoup
# Для генерации поддельного User agent
from fake_useragent import UserAgent
# Для работы с HTTP-запросами 
import requests
from requests import ConnectTimeout, ConnectionError, ReadTimeout 
from requests.exceptions import ProxyError
import urllib

# Для работы с табличными данными
import pandas as pd

# Для работы с регулярными выражениями 
import re

# Для работы с массивами и вычислениями
import numpy as np 

# Для работы с SQL
import sqlalchemy
from sqlalchemy import create_engine

# Для работы с операционной системой
import os

# Для работы с циклами
from itertools import cycle

# Для работы с математическими вычислениями
import math

# Для параллельной работы кода
from multiprocessing.dummy import Pool as ThreadPool 

# Для произведения синтаксического анализа (лемматизации)
import pymorphy2 as pm
# Загрузим словарь русского языка
morph = pm.MorphAnalyzer()

# Для работы со временем
import datetime

In [2]:
def address_coordinates_city(address): 
    """
    Функция для возвращения координат с yandex карт.
    Вход: адрес.
    Выход: долгота, широта. 
    """
    while True:     
        try: 
            url = 'https://yandex.ru/maps/75/vladivostok/?text={text}'.format(text = urllib.parse.quote(address))
            headers = {'User-Agent' : UserAgent().chrome}
            proxies = {'https' : 'https://' + next(proxy_cycle)}
            session = requests.Session()
            adapter = requests.adapters.HTTPAdapter(max_retries = 5)
            session.mount('https://', adapter)
            raw_coordinates = my_session(            
                url = url, 
                headers = headers, 
                proxies = proxies, 
                session = session
            )
            bsObj = BeautifulSoup(raw_coordinates.text, 'html5lib')
            coordinates_list = re.findall(r'"address":".+?","coordinates":\[\d{1,3}.\d{1,8},\d{1,3}.\d{1,8}\]', str(bsObj))[0]
            # Если не находим координаты
            if len(coordinates_list) is None  or len(coordinates_list) == 0:
                latitude, longitude = 0, 0
            # Если находим 1 значение
            elif isinstance('as', str): 
                coordinates_list = re.findall(r'\[\d{1,3}.\d{1,6}\,\d{1,3}.\d{1,8}\]', coordinates_list)[0]
                coordinates_list = re.sub(r'\[|\]', '', coordinates_list).split(',')
                longitude = coordinates_list[0]
                latitude = coordinates_list[1]
            else: 
                coordinates_list = re.findall(r'\[\d{1,3}.\d{1,6}\,\d{1,3}.\d{1,8}\]', coordinates_list[0])[0]
                coordinates_list = re.sub(r'\[|\]', '', coordinates_list).split(',')
                longitude = coordinates_list[0]
                latitude = coordinates_list[1]
            return [float(latitude), float(longitude)]
            break      
        except (ConnectTimeout, ProxyError, ConnectionError, ReadTimeout): 
            continue

def address_coordinates(address, city_latitude, city_longitude): 
    """
    Функция для возвращения координат с yandex карт.
    Вход: адрес.
    Выход: долгота, широта. 
    """
    while True:     
        try: 
            url = 'https://yandex.ru/maps/75/vladivostok/?text={text}'.format(text = urllib.parse.quote(address))
            headers = {'User-Agent' : UserAgent().chrome}
            proxies = {'https' : 'https://' + next(proxy_cycle)}
            session = requests.Session()
            adapter = requests.adapters.HTTPAdapter(max_retries = 5)
            session.mount('https://', adapter)
            raw_coordinates = my_session(            
                url = url, 
                headers = headers, 
                proxies = proxies, 
                session = session, 
                timeout = 10
            )
            bsObj = BeautifulSoup(raw_coordinates.text, 'html5lib')
            coordinates_list = re.findall(r'"address":".+?","coordinates":\[\d{1,3}.\d{1,8},\d{1,3}.\d{1,8}\]', str(bsObj))
            # Если не находим координаты
            if len(coordinates_list) == 0:
                latitude, longitude = 0, 0
            # Если находим 1 значение
            elif isinstance(coordinates_list, str): 
                coordinates_list = re.findall(r'\[\d{1,3}.\d{1,6}\,\d{1,3}.\d{1,8}\]', coordinates_list)[0]
                coordinates_list = re.sub(r'\[|\]', '', coordinates_list).split(',')
                longitude = coordinates_list[0]
                latitude = coordinates_list[1]
            else: 
                coordinates_list = re.findall(r'\[\d{1,3}.\d{1,6}\,\d{1,3}.\d{1,8}\]', coordinates_list[0])
                coordinates_list = re.sub(r'\[|\]', '', coordinates_list[0]).split(',')
                longitude = coordinates_list[0]
                latitude = coordinates_list[1]
            # Если координаты совпадают с координатами населенного пункта, то возвращаем None
            if float(longitude) == city_longitude and float(latitude) == city_latitude: 
                result_longitude, result_latitude = 0, 0            
            else: 
                result_longitude = float(longitude)
                result_latitude = float(latitude)
            return [result_latitude, result_longitude]
            break 
        except (ConnectTimeout, ProxyError, ConnectionError, ReadTimeout): 
            continue

In [ ]:
# Получаем координаты города Владивосток
city_latitude, city_longitude = address_coordinates_city('Приморский край, Владивосток')
print(city_latitude , city_longitude)